In [150]:
import pandas as pd
import gc

In [151]:
import numpy as np
import sys
import glob
sys.setrecursionlimit(9999)

In [152]:
csv_list = glob.glob('./HAI 2.0/training/*.csv')

In [153]:
train1 = pd.read_csv(csv_list[0]).rename(columns=lambda x: x.strip())
train2 = pd.read_csv(csv_list[1]).rename(columns=lambda x: x.strip())
train3 = pd.read_csv(csv_list[2]).rename(columns=lambda x: x.strip())

In [154]:
train = pd.concat([train1,train2,train3])
del train1 , train2, train3 
gc.collect()


24693

In [155]:
TAG_MIN = train[['C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'C09', 'C10',
       'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20',
       'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30',
       'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C40',
       'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'C49', 'C50',
       'C51', 'C52', 'C53', 'C54', 'C55', 'C56', 'C57', 'C58', 'C59', 'C60',
       'C61', 'C62', 'C63', 'C64', 'C65', 'C66', 'C67', 'C68', 'C69', 'C70',
       'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79']].min()
TAG_MAX = train[['C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'C09', 'C10',
       'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20',
       'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30',
       'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C40',
       'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'C49', 'C50',
       'C51', 'C52', 'C53', 'C54', 'C55', 'C56', 'C57', 'C58', 'C59', 'C60',
       'C61', 'C62', 'C63', 'C64', 'C65', 'C66', 'C67', 'C68', 'C69', 'C70',
       'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79']].max()

In [156]:
def normalize(df):
    ndf = df.copy()
    for c in df.columns:
        if TAG_MIN[c] == TAG_MAX[c]:
            ndf[c] = df[c] - TAG_MIN[c]
        else:
            ndf[c] = (df[c] - TAG_MIN[c]) / (TAG_MAX[c] - TAG_MIN[c])
    return ndf

In [157]:
train_scaled = normalize(train.drop('time', axis=1))
train_scaled = pd.DataFrame(train_scaled , columns=train.drop('time', axis=1).columns).ewm(alpha=0.9).mean()
train_scaled['time'] = train['time'].tolist()
train_scaled['time']= pd.to_datetime(train_scaled['time'])
train_scaled = train_scaled.sort_values('time')

In [158]:
from tqdm.notebook import trange
from datetime import timedelta
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import dateutil

WINDOW_GIVEN = 29
WINDOW_SIZE = 30


class HaiDataset(Dataset):
    def __init__(self, timestamps, df, stride=1, attacks=None):
        self.ts = np.array(timestamps)
        self.tag_values = np.array(df, dtype=np.float32)
        self.valid_idxs = []
        for L in trange(len(self.ts) - WINDOW_SIZE + 1):
            R = L + WINDOW_SIZE - 1
            if dateutil.parser.parse(self.ts[R]) - dateutil.parser.parse(
                self.ts[L]
            ) == timedelta(seconds=WINDOW_SIZE - 1):
                self.valid_idxs.append(L)
        self.valid_idxs = np.array(self.valid_idxs, dtype=np.int32)[::stride]
        self.n_idxs = len(self.valid_idxs)
        print(f"# of valid windows: {self.n_idxs}")
        if attacks is not None:
            self.attacks = np.array(attacks, dtype=np.float32)
            self.with_attack = True
        else:
            self.with_attack = False

    def __len__(self):
        return self.n_idxs

    def __getitem__(self, idx):
        i = self.valid_idxs[idx]
        last = i + WINDOW_SIZE - 1
        item = {"attack": self.attacks[last]} if self.with_attack else {}
        item["ts"] = self.ts[i + WINDOW_SIZE - 1]
        item["given"] = torch.from_numpy(self.tag_values[i : i + WINDOW_GIVEN])
        item["answer"] = torch.from_numpy(self.tag_values[last])
        return item

In [159]:
HAI_DATASET_TRAIN = HaiDataset(train['time'], train_scaled.drop('time',axis=1), stride=1)


# of valid windows: 921516


In [160]:
import torch

In [161]:
HAI_DATASET_TRAIN[0]['given'].numpy()

array([[0.37895253, 0.        , 0.        , ..., 0.2650172 , 1.        ,
        0.5672542 ],
       [0.3785037 , 0.        , 0.        , ..., 0.25179192, 1.        ,
        0.512135  ],
       [0.37846327, 0.        , 0.        , ..., 0.25470728, 1.        ,
        0.46962237],
       ...,
       [0.3824931 , 0.        , 0.        , ..., 0.19575165, 1.        ,
        0.6128688 ],
       [0.38153023, 0.        , 0.        , ..., 0.20010608, 1.        ,
        0.7011763 ],
       [0.38187853, 0.        , 0.        , ..., 0.19824855, 1.        ,
        0.6571191 ]], dtype=float32)

In [162]:
X_train =  np.array([i['given'].numpy() for i in HAI_DATASET_TRAIN ])
Y_train = np.array([i['answer'].numpy() for i in HAI_DATASET_TRAIN ])

In [163]:
X_train.shape

(921516, 29, 79)

In [164]:
val_csv = pd.read_csv('./HAI 2.0/validation/validation.csv')
test_csv_list = glob.glob('./HAI 2.0/testing/*.csv')

In [165]:
test1 = pd.read_csv(test_csv_list[0]).rename(columns=lambda x: x.strip())
test2 = pd.read_csv(test_csv_list[1]).rename(columns=lambda x: x.strip())
test3 = pd.read_csv(test_csv_list[2]).rename(columns=lambda x: x.strip())
test4 = pd.read_csv(test_csv_list[3]).rename(columns=lambda x: x.strip())

test_df = pd.concat([test1,test2,test3,test4])

In [166]:
del test1 , test2, test3, test4 
gc.collect()

20

In [167]:
val_csv = val_csv.sort_values('time')
test_df = test_df.sort_values('time')
val_scaled = normalize(val_csv.drop(['time','attack'] , axis=1))
val_scaled = pd.DataFrame(val_scaled , columns=val_csv.drop('time', axis=1).columns).ewm(alpha=0.9).mean()

In [168]:
HAI_DATASET_val = HaiDataset(val_csv['time'], val_scaled, stride=1)


# of valid windows: 43172


In [169]:
X_val =  np.array([i['given'].numpy() for i in HAI_DATASET_val ])
Y_val = np.array([i['answer'].numpy() for i in HAI_DATASET_val ])


In [170]:
test_scaled = normalize(test_df.drop('time' , axis=1))
test_scaled = pd.DataFrame(test_scaled , columns=test_df.drop('time', axis=1).columns).ewm(alpha=0.9).mean()
HAI_DATASET_test = HaiDataset(test_df['time'], test_scaled, stride=1)


# of valid windows: 358688


In [171]:
X_test =  np.array([i['given'].numpy() for i in HAI_DATASET_test ])
Y_test = np.array([i['answer'].numpy() for i in HAI_DATASET_test ])


In [172]:
import lightgbm as lgb
from sklearn.multioutput import MultiOutputRegressor

In [173]:
model = lgb.LGBMRegressor(random_state= 69)

In [174]:
X_train_reshape =X_train.reshape(len(X_train)  , WINDOW_GIVEN * 79 )
X_val_reshape =X_val.reshape(len(X_val) , WINDOW_GIVEN * 79 )
X_test_reshape = X_test.reshape(len(X_test) , WINDOW_GIVEN * 79 )



ValueError: cannot reshape array of size 100159040 into shape (43172,2291)

In [175]:
regressor_model = MultiOutputRegressor(model , n_jobs=-1).fit(X_train_reshape, Y_train)

In [176]:
from sklearn.tree import DecisionTreeClassifier

In [177]:
dt = DecisionTreeClassifier()

In [178]:
dt

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [179]:
regressor_model

MultiOutputRegressor(estimator=LGBMRegressor(boosting_type='gbdt',
                                             class_weight=None,
                                             colsample_bytree=1.0,
                                             importance_type='split',
                                             learning_rate=0.1, max_depth=-1,
                                             min_child_samples=20,
                                             min_child_weight=0.001,
                                             min_split_gain=0.0,
                                             n_estimators=100, n_jobs=-1,
                                             num_leaves=31, objective=None,
                                             random_state=69, reg_alpha=0.0,
                                             reg_lambda=0.0, silent=True,
                                             subsample=1.0,
                                             subsample_for_bin=200000,
                     

In [180]:
Y_val_predict = regressor_model.predict(X_val_reshape)

exception calling callback for <Future at 0x7fa12904bd10 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/opt/anaconda3/lib/python3.7/site-packages/joblib/parallel.py", line 340, in __call__
    self.parallel.dispatch_next()
  File "/opt/anaconda3/lib/python3.7/site-packages/joblib/parallel.py", line 769, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "/opt/anaconda3/lib/python3.7/site-packages/joblib/parallel.py", line 835, in dispatch_one_batch
    self._dispatch(tasks)
  File "/opt/anaconda3/lib/python3.7/site-packages/joblib/parallel.py", line 754, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "/opt/anaconda3/lib/python3.7/site-packages/joblib/_parallel_backends.py", line 551, in apply_async
    future = self._workers.submit(SafeFunction(func))

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker. The exit codes of the workers are {SIGABRT(-6), SIGABRT(-6)}

In [181]:
Y_val_predict.shape

(43172, 79)

In [182]:
error_val = abs(Y_val_predict - Y_val).sum(axis=1)

ValueError: operands could not be broadcast together with shapes (43172,79) (43172,80) 

In [183]:
pd.Series(error_val).plot()

RuntimeError: In set_text: could not load glyph

In [184]:
from joblib import dump, load
dump(regressor_model, 'xgb_60.joblib') 

['xgb_60.joblib']

In [185]:
label = pd.read_csv('./HAI 2.0/validation/validation.csv')['attack']

In [186]:
label[30:]

30       0
31       0
32       0
33       0
34       0
        ..
43196    0
43197    0
43198    0
43199    0
43200    0
Name: attack, Length: 43171, dtype: int64

In [187]:
error_val

array([1.33593992, 1.1178928 , 1.31626182, ..., 0.40475193, 0.53104122,
       0.70214024])

In [188]:
Y_test_hat = regressor_model.predict(X_test_reshape)

In [189]:
error_test = abs(Y_test_hat - Y_test).sum(axis=1)

In [190]:
pd.Series(error_test).plot()

RuntimeError: In set_text: could not load glyph

In [191]:
np.array(error_test).mean()

13.68925944689389

In [192]:
test_val= val_csv[['time','attack']]

In [193]:
test_val.shape

(43201, 2)

In [194]:
error_val.shape

(43172,)

In [195]:
test_val['dist'] = 0
test_val['dist'][29:] = error_val

In [196]:
test_val[29:][['dist']].describe()

,dist
count,43172.000000
mean,0.566439
std,0.529400
min,0.143458
25%,0.389408
50%,0.477701
75%,0.593786
max,10.477919


In [197]:
HAI_DATASET_val[-1]

{'ts': '2020-07-08 03:00:00',
 'given': tensor([[0.3977, 0.0000, 0.0000,  ..., 0.1797, 0.1277,    nan],
         [0.3964, 0.0000, 0.0000,  ..., 0.1797, 0.2357,    nan],
         [0.3967, 0.0000, 0.0000,  ..., 0.1797, 0.2937,    nan],
         ...,
         [0.3981, 0.0000, 0.0000,  ..., 0.1797, 0.2865,    nan],
         [0.3978, 0.0000, 0.0000,  ..., 0.1797, 0.2435,    nan],
         [0.3995, 0.0000, 0.0000,  ..., 0.1797, 0.1411,    nan]]),
 'answer': tensor([4.0102e-01, 0.0000e+00, 0.0000e+00, 6.9898e-01, 5.2645e-01, 6.1472e-04,
         4.7794e-01, 4.9326e-01, 5.7870e-04, 5.8319e-01, 4.9827e-01, 9.0758e-01,
         9.9456e-01, 0.0000e+00, 8.8746e-01, 3.2893e-01, 1.3668e-03, 0.0000e+00,
         0.0000e+00, 5.9375e-01, 0.0000e+00, 0.0000e+00, 8.9850e-01, 8.4808e-01,
         0.0000e+00, 8.6809e-01, 5.4577e-01, 1.0000e+00, 6.1807e-01, 1.1478e-01,
         7.3951e-01, 3.3411e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 7.9806e-01,
         0.0000e+00, 9.7120e-01, 4.3541e-01, 0.0000e+00, 3.

In [198]:
test_val[29:].groupby('attack').describe().unstack(1)

             attack
dist  count  0         42543.000000
             1           629.000000
      mean   0             0.525642
             1             3.325830
      std    0             0.225730
             1             2.841756
      min    0             0.143458
             1             0.262657
      25%    0             0.388170
             1             1.307399
      50%    0             0.475003
             1             1.935514
      75%    0             0.587571
             1             4.778329
      max    0             5.090029
             1            10.477919
dtype: float64

In [199]:
threshold = 4.602
test_val['real'] =test_val['dist'] > threshold
test_val['real'] = test_val['real'].astype(int)

In [200]:
from TaPR_pkg import etapr
TaPR = etapr.evaluate(anomalies=test_val['attack'].tolist(), predictions=test_val['real'].tolist())
print(f"F1: {TaPR['f1']:.3f} (TaP: {TaPR['TaP']:.3f}, TaR: {TaPR['TaR']:.3f})")
print(f"# of detected anomalies: {len(TaPR['Detected_Anomalies'])}")
print(f"Detected anomalies: {TaPR['Detected_Anomalies']}")

F1: 0.665 (TaP: 1.000, TaR: 0.499)
# of detected anomalies: 3
Detected anomalies: ['1', '4', '5']


In [201]:
submit = pd.read_csv('./HAI 2.0/sample_submission.csv')

In [202]:
error_test.shape

(358688,)

In [203]:
submit

,time,attack
0,2020-07-09 15:00:00,0
1,2020-07-09 15:00:01,0
2,2020-07-09 15:00:02,0
3,2020-07-09 15:00:03,0
4,2020-07-09 15:00:04,0
...,...,...
358799,2020-07-31 12:29:56,0
358800,2020-07-31 12:29:57,0
358801,2020-07-31 12:29:58,0
358802,2020-07-31 12:29:59,0


In [204]:
HAI_DATASET_test[0]

{'ts': '2020-07-09 15:00:29',
 'given': tensor([[0.2737, 0.0000, 0.0000,  ..., 0.2004, 0.2616, 0.7147],
         [0.2737, 0.0000, 0.0000,  ..., 0.1985, 0.2616, 0.6501],
         [0.2728, 0.0000, 0.0000,  ..., 0.1969, 0.2616, 0.6443],
         ...,
         [0.2634, 0.0000, 0.0000,  ..., 0.4188, 0.2616, 0.6507],
         [0.2625, 0.0000, 0.0000,  ..., 0.4259, 0.2616, 0.6417],
         [0.2624, 0.0000, 0.0000,  ..., 0.4354, 0.2616, 0.6981]]),
 'answer': tensor([ 2.6013e-01,  0.0000e+00,  0.0000e+00,  8.2833e-01,  1.8723e-01,
          9.9892e-04,  3.3616e-01,  3.5336e-01,  1.3310e-02,  1.3468e-01,
          4.9008e-01,  4.3796e-01,  7.8484e-01,  0.0000e+00,  8.9570e-01,
          1.5960e-01, -3.6563e-04,  0.0000e+00,  0.0000e+00,  3.6280e-01,
          0.0000e+00,  0.0000e+00,  4.1760e-04,  9.4967e-01,  0.0000e+00,
          8.7867e-01,  4.7043e-01,  0.0000e+00,  3.7816e-01,  9.2204e-02,
          9.8713e-01,  3.3413e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          1.0000e+00,  0.00

In [205]:
test_df

,time,C01,C02,C03,C04,C05,C06,C07,C08,C09,...,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79
0,2020-07-09 15:00:00,384.30737,12.0,10,70.35980,-1.6171,-1.79901,774.20752,28.02385,136,...,936.58447,0.0,1.35437,13.97231,35.22338,12.02545,293.51129,283.92651,32.0,6.5059
1,2020-07-09 15:00:01,384.30737,12.0,10,70.35980,-1.7606,-1.79901,772.58758,28.02730,136,...,940.93317,0.0,1.35437,13.93358,35.20813,12.02545,292.67938,283.36591,32.0,6.3079
2,2020-07-09 15:00:02,384.20517,12.0,10,70.35980,-1.7606,-1.80664,772.58758,28.02730,136,...,936.58447,0.0,1.35513,13.95248,35.20813,12.02545,291.90179,282.93189,32.0,6.3079
3,2020-07-09 15:00:03,384.25626,12.0,10,70.35980,-1.7814,-1.79901,777.48810,28.02905,136,...,933.54034,0.0,1.35513,13.89971,35.20813,12.02545,291.59430,282.06378,32.0,6.1203
4,2020-07-09 15:00:04,384.20517,12.0,10,70.35980,-1.7370,-1.79901,778.42212,28.03169,136,...,944.41223,0.0,1.35437,13.94603,35.20813,12.02545,289.87628,283.67334,32.0,5.9543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92396,2020-07-31 12:29:56,420.08923,12.0,10,48.31848,-0.8706,97.19238,786.54382,28.03253,232,...,824.82147,100.0,1.35666,9.62203,36.47460,11.78894,357.27722,361.14728,32.0,6.2809
92397,2020-07-31 12:29:57,420.08923,12.0,10,48.31848,-0.7498,97.19238,784.07184,28.03598,224,...,823.51697,100.0,1.35513,9.48747,36.47460,11.78894,357.29529,359.84521,32.0,6.3602
92398,2020-07-31 12:29:58,420.24258,12.0,10,48.31848,-0.6076,97.19238,786.83881,28.02642,208,...,824.82147,100.0,1.35666,9.57787,36.48986,11.78894,357.27722,360.60474,32.0,6.3742
92399,2020-07-31 12:29:59,420.24258,12.0,10,48.31848,-0.4618,97.19238,786.66138,28.03341,200,...,833.51904,100.0,1.35513,9.56291,36.48986,11.78894,357.80170,357.42188,32.0,6.2864


In [206]:
val_csv

,time,C01,C02,C03,C04,C05,C06,C07,C08,C09,...,C71,C72,C73,C74,C75,C76,C77,C78,C79,attack
0,2020-07-07 15:00:00,402.70947,12.0,10,51.95007,-1.0189,-1.86768,789.76508,28.03162,688,...,0.0,1.34293,10.89290,34.88770,12.26196,380.31683,386.26666,32.59527,5.6330,0
1,2020-07-07 15:00:01,402.81174,12.0,10,51.96533,-1.2637,-1.86768,789.13147,28.02301,648,...,0.0,1.34216,10.80512,34.88770,12.26196,380.02747,386.30286,32.59527,5.4158,0
2,2020-07-07 15:00:02,402.76062,12.0,10,51.96533,-1.5398,-1.86768,785.81653,28.02993,616,...,0.0,1.34369,10.80029,34.88770,12.26196,381.52850,389.73883,32.59527,5.5532,0
3,2020-07-07 15:00:03,402.81174,12.0,10,51.98822,-1.6212,-1.86768,785.42438,28.02993,584,...,0.0,1.34445,10.80579,34.88770,12.26196,382.08911,388.94311,32.59527,5.7833,0
4,2020-07-07 15:00:04,402.91394,12.0,10,51.90429,-1.5631,-1.86768,782.99249,28.02990,552,...,0.0,1.34293,10.81415,34.90295,12.26196,383.44543,389.72082,32.59527,6.0309,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43196,2020-07-08 02:59:56,397.08661,12.0,10,66.58325,-1.2052,-1.83716,786.93738,28.03250,0,...,0.0,1.35971,16.19496,35.22338,12.01019,390.13672,394.91107,31.81634,5.2977,0
43197,2020-07-08 02:59:57,397.18887,12.0,10,66.58325,-0.9256,-1.83716,783.44989,28.02304,0,...,0.0,1.35971,16.23927,35.23864,12.01019,390.24518,397.35248,31.81634,5.3188,0
43198,2020-07-08 02:59:58,397.13776,12.0,10,66.58325,-0.7843,-1.83716,784.86780,28.02814,0,...,0.0,1.35818,16.20675,35.23864,12.01019,390.46222,396.70142,31.81634,5.1800,0
43199,2020-07-08 02:59:59,397.34222,12.0,10,66.58325,-0.7646,-1.83716,785.51416,28.02294,0,...,0.0,1.35818,16.17168,35.25391,12.01019,391.78241,397.73218,31.81634,4.8763,0
